<a href="https://colab.research.google.com/github/jermwatt/morphi_lab/blob/object_diffusion_collab_demo/object_diffusion_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single object diffusion demo

This notebook allows for experimentation with object detection + segmentation --> diffusion.

To use it first:

- select a gpu runtime: make sure you have chosen a GPU runtime from the options above - choose A100 if available.
- activate the first few cells to 
  - perform a machine check: make sure your runtime is employing a GPU with sufficient memory
  - download associated files and installs
- start experimenting!


## 1.  machine setup and installs

### 1.1. pre-launch machine check

Activate the code block below to double check that you are using a GPU runtime for the experiments.  You don't have to use one if you can't, but the experiments will run significantly faster (in particular the diffusion steps) if you are using one.

In [ ]:
# check for GPU runtime
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# check for memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

### 1.2. installs

Activate the cell below to install all necessary packages to your collab enviroment for these experiments.

In [ ]:
# install reqiured libraries 
!pip install "ultralytics==8.0.111" "transformers==4.29.2" "timm==0.9.2" "diffusers==0.16.1" "safetensors==0.3.1" "accelerate==0.19.0"

Activate the cell below to pull all helper functions required to run this demo.

In [ ]:
# pull some images from the web
import urllib.request

def download_file(url, output_path):
    urllib.request.urlretrieve(url, output_path)

# pull segmentation module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/segmenter.py"
local_path = "/content/segmenter.py"
download_file(url,local_path)

# pull diffusion module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/diffuser.py"
local_path = "/content/diffuser.py"
download_file(url,local_path)

# pull utilities module
url = "https://raw.githubusercontent.com/jermwatt/morphi_lab/object_diffusion_collab_demo/utilities.py"
local_path = "/content/utilities.py"
download_file(url,local_path)

Activate the cell below to pull in a a sample images to experiment with.

In [ ]:
# man holding donut - we'll use this one for testing
url = "https://www.shutterstock.com/image-photo/surprised-young-man-holding-donut-260nw-586330142.jpg"
output_path = "/content/test_donut.png"
download_file(url, output_path)

### 1.3.  Module setup

Activate the next cell to load in the required functionality from your newly installed modules.

The first time you do this both a Yolo and Diffusion models (so will take 30 seconds to a minute or two to execute).

In [ ]:
# import segmentation and diffusion modules
from segmenter import segment_image, label_lookup_dict
from diffuser import diffuse_segmented_img
from utilities import show_img, show_all_results

## 2. start experimenting!

Lets start by looking at the image we downloaded.

In [ ]:
# plot the downloaded test image
show_img('/content/test_donut.png')

### 2.1. Segmentation

All right.  Let's segment this image.

This demo has all of the pre-trained yolo labels available for segmentation.

To see all of the currently available labels activate the cell below.

In [ ]:
# print out all of the available labels for segmentation
list(label_lookup_dict.keys())

Since `person` and `donut` are in this image, let's segment each separately.

Remember: in this demo you can only segment one distinct object per execution (e.g., `person` OR `donut`).

First the donut.

In [ ]:
# segment the donut out of the test image
img_path = "/content/test_donut.png"
labels = ['donut']
img, mask, seg = segment_image(img_path,
                               labels=labels)

In [ ]:
# show the segmented image - here the donut is masked with "pink"
seg.show_result()

Let's segment the same image for the person.

In [ ]:
# segment the donut out of the test image
img_path = "/content/test_donut.png"
labels = ['person']
img, mask, seg = segment_image(img_path,
                               labels=labels)

In [ ]:
# show the segmented image - here the person is masked with "pink"
seg.show_result()

### 2.3. diffuse an object

After segmenting, we can pass our image and segmented mask to the diffuser.

Let's try this below with our `person` segmentation and prompt about an ape.

In [ ]:
# segment the donut out of the test image
img_path = "/content/test_donut.png"
labels = ['person']
img, mask, seg = segment_image(img_path,
                               labels=labels)

# diffuse the masked segmentation 
diffused_img = diffuse_segmented_img(img,
                                     mask,
                                     prompt='an ape, smiling, high resolution, holding something',
                                     seed=3433)

# show results
show_all_results(seg.orig_img,
                 mask,
                 diffused_img)

Notice the fixed `seed` I am inputing (to keep the output look the same).  If you want a random seed - simply omit this option.

Let's try this below.

In [ ]:
# segment the donut out of the test image
img_path = "/content/test_donut.png"
labels = ['person']
img, mask, seg = segment_image(img_path,
                               labels=labels)

# diffuse the masked segmentation 
diffused_img = diffuse_segmented_img(img,
                                     mask,
                                     prompt='an ape, smiling, high resolution, holding something')

# show results
show_all_results(seg.orig_img,
                 mask,
                 diffused_img)

Let's try the same image - different object.

Now let's replace the `donut`. 

In [ ]:
# segment the donut out of the test image
img_path = "/content/test_donut.png"
labels = ['donut']
img, mask, seg = segment_image(img_path,
                               labels=labels)

# diffuse the masked segmentation 
diffused_img = diffuse_segmented_img(img,
                                     mask,
                                     prompt='an orange, fruit, high resolution',
                                     seed=3433)

# show results
show_all_results(seg.orig_img,
                 mask,
                 diffused_img)